In [9]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

### IUCN (integrando via API, para informações faltantes)

In [12]:
conn.execute("ALTER TABLE iucn ADD COLUMN redListCriteriaCode TEXT")
conn.execute("ALTER TABLE iucn ADD COLUMN extant BOOLEAN DEFAULT FALSE")
conn.execute("ALTER TABLE iucn ADD COLUMN criteria TEXT DEFAULT NULL")

In [13]:
res = conn.execute("SELECT * FROM iucn").df()
res.head()

,internalTaxonId,scientificName_x,kingdomName,phylumName,className,orderName,familyName,genusName,speciesName,infraType,...,systems,conservationActions,realm,yearLastSeen,possiblyExtinct,possiblyExtinctInTheWild,scopes,redListCriteriaCode,extant,criteria
0,30646,Piper distigmatum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,distigmatum,NaN,...,Terrestrial,None,Neotropical,NaN,False,False,Global,None,False,None
1,35768,Piper verrucosum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,verrucosum,NaN,...,Terrestrial,None,Neotropical,NaN,False,False,Global,None,False,None
2,44006,Piper seychellarum,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,seychellarum,NaN,...,Terrestrial,Forest areas are managed for conservation by t...,Afrotropical,NaN,False,False,Global,None,False,None
3,45860,Piper san-miguelense,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,san-miguelense,NaN,...,Terrestrial,None,Neotropical,NaN,False,False,Global,None,False,None
4,45862,Piper skutchii,PLANTAE,TRACHEOPHYTA,MAGNOLIOPSIDA,PIPERALES,PIPERACEAE,Piper,skutchii,NaN,...,Terrestrial,None,Neotropical,NaN,False,False,Global,None,False,None


In [21]:
conn.execute("DROP TABLE IF EXISTS iucn_complete")
conn.execute("CREATE TABLE iucn_complete AS SELECT * FROM iucn LIMIT 0;")

In [34]:
for _, row in res.iterrows():
    url = "https://api.iucnredlist.org/api/v4/assessment/%d" % row["assessmentId"]
    response = make_requests(url, headers={"Authorization": os.environ["IUCN_API"]})
    if response and "locations" in response:
        print(response["locations"])
        a = row['internalTaxonId']
        print(a)
        # sql = f"""
        # INSERT INTO iucn_complete
        # VALUES ()
        # """
    break

[{'is_endemic': True, 'formerlyBred': None, 'origin': 'Native', 'presence': 'Extant', 'seasonality': None, 'description': {'en': 'Panama'}, 'code': 'PA'}]
30646
[{'is_endemic': True, 'formerlyBred': None, 'origin': 'Native', 'presence': 'Extant', 'seasonality': None, 'description': {'en': 'Jamaica'}, 'code': 'JM'}]
35768
[{'is_endemic': True, 'formerlyBred': None, 'origin': 'Native', 'presence': 'Extant', 'seasonality': None, 'description': {'en': 'Seychelles'}, 'code': 'SC'}]
44006
[{'is_endemic': True, 'formerlyBred': None, 'origin': 'Native', 'presence': 'Extant', 'seasonality': None, 'description': {'en': 'Ecuador'}, 'code': 'EC'}]
45860
[{'is_endemic': True, 'formerlyBred': None, 'origin': 'Native', 'presence': 'Extant', 'seasonality': None, 'description': {'en': 'Ecuador'}, 'code': 'EC'}]
45862
[{'is_endemic': True, 'formerlyBred': None, 'origin': 'Native', 'presence': 'Extant', 'seasonality': None, 'description': {'en': 'Ecuador'}, 'code': 'EC'}]
45861
[{'is_endemic': True, 'for

ValueError: 

In [8]:
conn.close()